In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

# 1. 导入必要的库

In [2]:
# 2. 定义数据加载器
batch_size = 64

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# 使用 归一化，training: 4m 15s, Accuracy:  99%, 不使用： training time:3m 52s, Accuracy: 99%
train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [3]:

# 3. 定义 FCN 模型
class FCN(nn.Module):
    def __init__(self):
        super(FCN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.conv4 = nn.Conv2d(128, 10, kernel_size=1)  # 输出通道数设为类别数（10）

        #for linear demo
        # self.out = nn.Linear(64*7*7,10)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = self.conv4(x)  # 不需要激活函数，直接进行最后的卷积
        x = x.mean([2, 3])  # 全局平均池化

        #for linear demo
        # x = x.view(x.size(0),-1)
        # x = self.out(x)
        return x

model = FCN()


# 4. 定义损失函数和优化器
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 5. 训练模型
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')

print('Finished Training')


Epoch [1/10], Loss: 0.3208
Epoch [2/10], Loss: 0.0788
Epoch [3/10], Loss: 0.0541
Epoch [4/10], Loss: 0.0427
Epoch [5/10], Loss: 0.0353
Epoch [6/10], Loss: 0.0301
Epoch [7/10], Loss: 0.0258
Epoch [8/10], Loss: 0.0222
Epoch [9/10], Loss: 0.0193
Epoch [10/10], Loss: 0.0158
Finished Training


In [4]:
# 6. 评估模型
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the model on the test images: {100 * correct / total:.2f}%')

Accuracy of the model on the test images: 99.09%
